In [2]:
import pandas as pd
import os

In [3]:
#cancer_df = pd.read_csv('/data/team2/top10data/cancer.csv')
#cancer_df['PMID'] = cancer_df['PMID'].apply(lambda x: x.split(','))
#exploded_df = cancer_df.explode('PMID')
#exploded_df = exploded_df.drop_duplicates('PMID')

'''
Files
* API_results_top_bottom_10.csv - this contains all PMIDs returned for a given query
* top10data/<query term>.csv - this contains data from users for a given query
* pmid_fulltext_status.csv - this contains PMIDs and whether they have full text available
* top_10_query_PMIDs_v2.csv - this contains metadata for PMIDs in returned by the top 10 queries

Each of these
'''

processed_query = 'cancer'

api_res_df = pd.read_csv('/data/team2/API_results_top_bottom_10.csv').rename(columns={'Unnamed: 0':'processed_query'})
pq_pmids = api_res_df[api_res_df['processed_query'] == processed_query]

# currently don't have metadata for all PMIDs in the API results dataframe
top_10_query_PMIDs = pd.read_csv('/data/team2/top_10_query_PMIDs_v2.csv').drop('Unnamed: 0', axis=1)
top_10_query_PMIDs['PMID'] = top_10_query_PMIDs['pmid'].astype(str)

# adding hasAbstract, hasStructuredAbstract, pubtype
merged_df = exploded_df.merge(top_10_query_PMIDs, how='left', on = 'PMID')

# adding one-hot encoded publication types
merged_df1 = merged_df.join(pd.get_dummies(merged_df.pubtype, prefix='pubtype'))

# adding full-text availability
fulltext = pd.read_csv('/data/team2/pmid_fulltext_status.csv').rename(columns={'Unnamed: 0':'PMID'})
fulltext['PMID'] = fulltext['PMID'].astype(str)
merged_df2 = merged_df1.merge(fulltext, on='PMID')
fulltext_dummy = pd.get_dummies(merged_df2.fulltext_status)
merged_df2['fulltext_status'] = fulltext_dummy['fulltext_available'] # 1 if it has fulltext, 0 if it doesn't

In [12]:
fulltext = pd.read_csv('/data/team2/pmid_fulltext_status.csv').rename(columns={'Unnamed: 0':'PMID'})
fulltext['PMID'] = fulltext['PMID'].astype(str)

In [13]:
merged_df2 = merged_df1.merge(fulltext, on='PMID')

In [16]:
fulltext_dummy = pd.get_dummies(merged_df2.fulltext_status)

In [17]:
merged_df2['fulltext_status'] = fulltext_dummy['fulltext_available'] # 1 if it has fulltext, 0 if it doesn't

In [19]:
merged_df2.shape

(573, 294)

In [21]:
merged_df2.head()

,search_id,query_term,filters,sort_algorithm,result_count,page_num,PMID,click_data,date,user_id,...,pubtype_['Journal Article'],"pubtype_['Letter', 'Comment']","pubtype_['Letter', 'Randomized Controlled Trial']",pubtype_['Letter'],"pubtype_['Meta-Analysis', ""Research Support, Non-U.S. Gov't"", 'Systematic Review']","pubtype_['News', 'Comment']",pubtype_['News'],pubtype_['Published Erratum'],pubtype_['Retraction of Publication'],fulltext_status
0,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,31761807,NoClicks,2022-03-16,Z392386,...,1,0,0,0,0,0,0,0,0,0
1,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,28244479,NoClicks,2022-03-16,Z392386,...,0,0,0,0,0,0,0,0,0,1
2,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,27741350,NoClicks,2022-03-16,Z392386,...,0,0,0,0,0,0,0,0,0,0
3,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,30713326,NoClicks,2022-03-16,Z392386,...,0,0,0,0,0,0,0,0,0,1
4,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,29949179,NoClicks,2022-03-16,Z392386,...,0,0,0,0,0,0,0,0,0,1


In [7]:
cancer_df.head()

,search_id,query_term,filters,sort_algorithm,result_count,page_num,PMID,click_data,date,user_id,filters_active,processed_query
0,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,"[31761807, 28244479, 27741350, 30713326, 29949...",NoClicks,2022-03-16,Z392386,False,cancer
1,5cr7r_f0i_6wyd0xw3ayza:93764b25ae99d315f410802...,cancer,None,date,4569599,1,"[35289556, 35289548, 35289536, 35289535, 35289...",NoClicks,2022-03-16,Z401427,False,cancer
2,_hyofx-kvkxkzrumj0tvoa:a99dca5593185c498b63a5e...,cancer,None,relevance,4569599,1,"[31761807, 28244479, 27741350, 30713326, 29949...","*result_click,28244479,2022-03-16T20:03:49,1,F...",2022-03-16,Z272979,False,cancer
3,_wbpom97rhxkmccuzyb26g:7463543d784aa59ca86359a...,cancer,None,relevance,4569599,1,"[31761807, 28244479, 27741350, 30713326, 29949...",NoClicks,2022-03-16,Z93392,False,cancer
4,aajrft_asz6yj4tjzrqkdq:26985a88de1e140c5bce6ac...,cancer,datesearch.y_5,relevance,1064948,1,"[29860986, 31761807, 30445141, 28154351, 30713...","*result_click,31761807,2022-03-16T20:44:35,1,F...",2022-03-16,Z292543,True,cancer


In [22]:
api_res = pd.read_csv('/data/team2/API_results_top_bottom_10.csv').rename(columns={'Unnamed: 0':'processed_query'})

In [26]:
bc_list = api_res.iloc[0,1]

In [27]:
len(bc_list)

117227

In [30]:
top_10_query_PMIDs.shape

(16819, 11)

In [33]:
top_10_query_PMIDs.head()

,pmid,journal,title,abstract,hasAbstract,hasStructuredAbstract,articleDate,journalPubDate,entrezDate,pubtype,PMID
0,35252802,Journal of biosafety and biosecurity,Recent developments and trends of automatic nu...,"Nucleic acid detection, widely used in clinica...",True,False,2022-02-28,NaN,2022-03-07,"['Journal Article', 'Review']",35252802
1,35287656,BMC public health,Nutrition standards for the charitable food sy...,Food insecurity is associated with increased r...,True,False,2022-03-14,2022-03-14,2022-03-15,"['Journal Article', ""Research Support, Non-U.S...",35287656
2,35232441,BMC medical education,Establishing CREATE: lessons learned in settin...,"[{'@Label': 'BACKGROUND', '@NlmCategory': 'BAC...",True,True,2022-03-01,2022-03-01,2022-03-02,['Journal Article'],35232441
3,32282054,Turk patoloji dergisi,Pulmonary Acantholytic Squamous Cell Carcinoma...,NaN,False,False,NaN,NaN,2020-04-14,['Journal Article'],32282054
4,33629745,Journal of cellular physiology,TRIM11 suppresses ferritinophagy and gemcitabi...,Gemcitabine is first-line chemotherapy for pan...,True,False,2021-02-25,NaN,2021-02-25,"['Journal Article', ""Research Support, Non-U.S...",33629745


In [31]:
cancer_df.shape

(439, 12)

In [32]:
cancer_df.head()

,search_id,query_term,filters,sort_algorithm,result_count,page_num,PMID,click_data,date,user_id,filters_active,processed_query
0,4ibte5dnudz3x_oeqibxuw:6dd943ee022371df5a9d5f0...,cancer,None,relevance,4569599,1,"[31761807, 28244479, 27741350, 30713326, 29949...",NoClicks,2022-03-16,Z392386,False,cancer
1,5cr7r_f0i_6wyd0xw3ayza:93764b25ae99d315f410802...,cancer,None,date,4569599,1,"[35289556, 35289548, 35289536, 35289535, 35289...",NoClicks,2022-03-16,Z401427,False,cancer
2,_hyofx-kvkxkzrumj0tvoa:a99dca5593185c498b63a5e...,cancer,None,relevance,4569599,1,"[31761807, 28244479, 27741350, 30713326, 29949...","*result_click,28244479,2022-03-16T20:03:49,1,F...",2022-03-16,Z272979,False,cancer
3,_wbpom97rhxkmccuzyb26g:7463543d784aa59ca86359a...,cancer,None,relevance,4569599,1,"[31761807, 28244479, 27741350, 30713326, 29949...",NoClicks,2022-03-16,Z93392,False,cancer
4,aajrft_asz6yj4tjzrqkdq:26985a88de1e140c5bce6ac...,cancer,datesearch.y_5,relevance,1064948,1,"[29860986, 31761807, 30445141, 28154351, 30713...","*result_click,31761807,2022-03-16T20:44:35,1,F...",2022-03-16,Z292543,True,cancer


In [35]:
api_res.head()

,Unnamed: 0,relevance_res,date_desc_res,query_type
0,breast cancer,"['29651637', '25456291', '31398351', '33786630...","['35612777', '35612761', '35612745', '35612727...",top_10
1,alzheimer's disease,"['27570871', '28269772', '27179961', '28211812...","['35612802', '35612780', '35612774', '35612546...",top_10
2,cancer,"['27839715', '31761807', '19491253', '26667886...","['35612878', '35612872', '35612856', '35612854...",top_10
3,lung cancer,"['34895608', '29157380', '28653990', '35263397...","['35612872', '35612853', '35612707', '35612630...",top_10
4,obesity,"['20871401', '15245383', '28292619', '7013664'...","['35612892', '35612619', '35612570', '35612502...",top_10


In [42]:
for i in range(api_res.shape[0]):
    print(api_res.iloc[i,0])
    print(len(api_res.iloc[i,1]))
    print(len(api_res.iloc[i,2]))
    print('\n')

breast cancer
117227
119988


alzheimer's disease
117719
119988


cancer
116963
119988


lung cancer
116731
119988


obesity
117368
119988


multiple sclerosis
117606
119988


diabetes mellitus
114103
119988


gastric cancer
115903
119988


ferroptosis
49080
49080


covid-19
119988
119988


allergies
116800
119988


cardiac ischemia
114415
119988


neurodegenerative disease
119709
119988


depression in older adults
5078
5078


adverse drug reactions
117362
119988


neuroinflammation
117402
119988


vasculitis
116335
119988


hypoxia
114963
119988


acute kidney injury
116324
119988


pulmonary embolism
114949
119988


